## Data Scraping
We are now on 2024, I decide to perform a regular collect of coronavirus for each month in order to build a prediction after 1 year some forecast and compare the models.. Please note that this data was scrapped from [worldometers.info/coronavirus](https://www.worldometers.info/coronavirus/). This data was solely for education purposes only. But for my own experience i tried to gather more data to make more analysis.

![Covid](https://storage.googleapis.com/kaggle-datasets-images/2513794/4266054/c2b54dc342fea08b4740760e1630cdbe/dataset-cover.jpg?t=2022-09-30-04-13-06)

We are now on 2024, I decide to perform a regular collect of coronavirus for each month in order to build a prediction after 1 year some forecast and compare the models.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sb
import requests
import re
from bs4 import BeautifulSoup as bf

In [2]:
url = 'https://www.worldometers.info/coronavirus/'
response = requests.get(url)
response

<Response [200]>

In [3]:
soup = bf(response.content, 'lxml')
"""with open('Coronavirus_24.html', mode = 'wb')as file:
    file.write(response.content)"""
bloc_countries = soup.find('table',class_="table table-bordered table-hover main_table_countries").contents[3]

In [4]:

#Id in tab 
id_case = bloc_countries.findAll('td', style="font-size:12px;color: grey;text-align:center;vertical-align:middle;")
id_list = [i.contents[0] for i in id_case]

#Country Cases
country_case = bloc_countries.findAll('td', style="font-weight: bold; font-size:15px; text-align:left;")
country_list = [i.contents[0].contents[0] for i in country_case]

#Total_Cases
total_cases_case = bloc_countries.findAll('td', style="font-weight: bold; text-align:right")
total_case_index = [i for i in range(len(total_cases_case)) if i%11 == 0]
list_searched = [total_cases_case[i] for i in total_case_index]
total_cases_list = [i.contents[0] for i in list_searched[:231]]


In [5]:
'''Fuction to convert the website into string 
and filter out contents since it's not possible effectively on some columns'''
#I'm obliged to add manually new cases in dataset , no way to extract directly content
def worldometers_to_string(bloc_countries):
    df_countries = []
    bloc_countries = str(bloc_countries)
    parts = re.findall(r'''(\n<td style="font-size:12px;color: grey;text-align:center;vertical-align:middle;">[\d]*</td>\n[\s\S]*?\n<!-- 1 Case/Death/Test every X -->\n)''',bloc_countries)
    print(len(parts))
    for i in range(len(parts)):
        #Id in tab
        Id_str = re.findall(r'''(grey;text-align:center;vertical-align:middle;">[\d,]+</td>)''',parts[i])
        Id_from_str = re.findall('[\d]+',Id_str[0])[0]
        #Id_from_str

        #Country in tab
        country_str = re.findall(r'''(<a class="mt_a" href="country/[a-zA-Z"/-]+>[a-zA-Z .]+</a>)''',parts[i])
        try:
            country_from_str = re.findall('[a-zA-Z .]+(?#</a>$)',country_str[0])[-2]
        except:
            country_from_str =None
        #country_from_str

        #Total_cases in tab
        total_cases_str = re.findall(r'''(<td style="font-weight: bold; text-align:right">[0-9,]+</td>)''',parts[i])
        try:
            total_cases_from_str = re.findall('[0-9,]+(?#</d>$)',total_cases_str[0])[0]
        except:
            total_cases_from_str = 0
        #total_cases_from_str
    
        #New_cases in tab
        new_cases_str = re.findall(r'''(text-align:right;background-color:#FFEEAA;">[0-9,+]+</td>)''',parts[i])
        try:
            new_cases_from_str = re.findall('[0-9,]+(?#</d>$)',new_cases_str[0])[0]
        except:
            new_cases_from_str = 0
        #new_cases_from_str

        #Total_Deaths in tab
        total_death_str = re.findall(r'''(<td style="font-weight: bold; text-align:right;">[0-9, ]+</td>)''',parts[i])
        try:
            total_death_from_str = re.findall('[0-9,]+(?#</d>$)',total_death_str[0])[0]
        except:
            total_death_from_str =0 
        #total_death_from_str

        #New_deaths in tab
        new_deaths_str = re.findall(r'''(background-color:red; color:white">[0-9,+]+</td>)''',parts[i])
        try:
            new_deaths_from_str = re.findall('[0-9,]+(?#</d>$)',new_deaths_str[0])[-1]
        except:
            new_deaths_from_str = 0
        #new_deaths_from_str

        #Total_Recovered in tab
        total_recovered_str = re.findall(r'''(<span style="color:grey; font-style: italic;">[0-9, ]+</span>)''',parts[i])
        #Affect the extraction of serious_critical cases
        if len(total_recovered_str) == 0:
            try:
                total_recovered_str = re.findall(r'''(<td style="font-weight: bold; text-align:right">[0-9, ]+</td>)''',parts[i])
                total_recovered_from_str = re.findall('[0-9,]+(?#</d>$)',total_recovered_str[1])[0]
            except:
                total_recovered_from_str=0
        else:
            total_recovered_from_str = re.findall('[0-9,]+(?#</d>$)',total_recovered_str[0])[0]
        #total_recovered_from_str

        #New_recovered in tab 
        new_recovered_str = re.findall(r'''(background-color:#c8e6c9; color:#000">[0-9,+]+</td>)''',parts[i])
        try:
            new_recovered_from_str = re.findall('[0-9,]+(?#</d>$)',new_recovered_str[0])[-1]
        except:
            new_recovered_from_str =0
        #new_recovered_from_str

        #Active_cases in tab 
        active_cases_str = re.findall(r'''(<td style="text-align:right;font-weight:bold;">[0-9,]+</td>)''',parts[i])
        try:
            active_cases_from_str = re.findall('[0-9,]+(?#</d>$)',active_cases_str[0])[-1]
        except:
            active_cases_from_str = 0
        #active_cases_from_str

        #Serious_critical, total_cases_per_million, deaths_per_million, tests_per_million in tab (based on this extraction method, depend of total recovered)
        total_recovered_str = re.findall(r'''(<span style="color:grey; font-style: italic;">[0-9, ]+</span>)''',parts[i])
        pattern_search = re.findall(r'''(<td style="font-weight: bold; text-align:right">[0-9,]+</td>)''',parts[i])
        serious_critical_str = re.findall(r'''(<td style="font-weight: bold; text-align:right">[0-9,]+</td>)''',parts[i])
        total_cases_per_million_str = pattern_search
        deaths_per_million_str = pattern_search
        total_tests_str = pattern_search
        tests_per_million_str = pattern_search
        if len(total_recovered_str) == 0:
            total_recovered_str = re.findall(r'''(<td style="font-weight: bold; text-align:right">[0-9, ]+</td>)''',parts[i])
            try:
                serious_critical_from_str = re.findall('[0-9,]+(?#</d>$)',serious_critical_str[2])[0]
            except:
                serious_critical_from_str = 0
            try:
                total_cases_per_million_from_str = re.findall('[0-9,]+(?#</d>$)',total_cases_per_million_str[3])[0]
            except:
                total_cases_per_million_from_str =0
            try:
                deaths_per_million_from_str = re.findall('[0-9,]+(?#</d>$)',total_cases_per_million_str[4])[0]
            except:
                deaths_per_million_from_str = 0
            try:
                total_tests_from_str = re.findall('[0-9,]+(?#</d>$)',total_cases_per_million_str[5])[0]
            except:
                total_tests_from_str = 0
            try:
                tests_per_million_from_str = re.findall('[0-9,]+(?#</d>$)',total_cases_per_million_str[6])[0]
            except:
                tests_per_million_from_str = 0
        else:
            try:
                serious_critical_from_str = re.findall('[0-9,]+(?#</d>$)',serious_critical_str[1])[0]
            except:
                serious_critical_from_str = 0
            try:
                total_cases_per_million_from_str = re.findall('[0-9,]+(?#</d>$)',total_cases_per_million_str[2])[0]
            except:
                total_cases_per_million_from_str =0
            try:
                deaths_per_million_from_str = re.findall('[0-9,]+(?#</d>$)',total_cases_per_million_str[3])[0]
            except:
                deaths_per_million_from_str = 0
            try:
                total_tests_from_str = re.findall('[0-9,]+(?#</d>$)',total_cases_per_million_str[4])[0]
            except:
                total_tests_from_str = 0
            try:
                tests_per_million_from_str = re.findall('[0-9,]+(?#</d>$)',total_cases_per_million_str[5])[0]
            except:
                tests_per_million_from_str = 0
    
            #Population in tab
        population_str = re.findall(r'''(text-align:right"><a href="/world-population/[a-zA-Z"/-]+>[0-9,]+</a>)''',parts[i])
        try :
            population_from_str = re.findall('[0-9,]+(?#</a>$)',population_str[0])[0]
        except:
            population_from_str = None
        #population_from_str

        #Continent in Worldometer tab sheet
        continent_str = re.findall(r'''(<td data-continent[a-zA-Z": =]+>[a-zA-Z. ]+</td>)''',parts[i])
        try:
            continent_from_str = re.findall('[a-zA-Z. ]+(?#</td>$)',continent_str[0])[-2]
        except:
            continent_from_str = None
        #continent_from_str
    

        #DataFrame building 
        df_countries.append({'Id':Id_from_str,
                         'country':country_from_str,
                         'total_cases':total_cases_from_str,
                         'new_cases':new_cases_from_str,
                         'total_death':total_death_from_str,
                         'new_deaths':new_deaths_from_str,
                         'total_recovered':total_recovered_from_str,
                         'new_recovered':new_recovered_from_str,
                         'active_cases':active_cases_from_str,
                         'serious_critical':serious_critical_from_str,
                         'total_cases_per_million':total_cases_per_million_from_str,
                         'deaths_per_million':deaths_per_million_from_str,
                         'total_tests':total_tests_from_str,
                         'tests_per_million':tests_per_million_from_str,
                         'population':population_from_str,
                         'continent':continent_from_str})

    df_coronavirus = pd.DataFrame(df_countries, columns = ['Id','country','total_cases','new_cases','total_death','new_deaths','total_recovered','new_recovered','active_cases','serious_critical','total_cases_per_million','deaths_per_million','total_tests','tests_per_million','population','continent'])
    return df_coronavirus

In [6]:
df = worldometers_to_string(bloc_countries)

231


In [13]:
from datetime import datetime
tdy = datetime.today().strftime('%Y-%m-%d')
df['datetime'] = tdy

In [14]:
df.to_csv('Coronavirus_'+tdy+'.csv',index =False)

In [15]:
df

,Id,country,total_cases,new_cases,total_death,new_deaths,total_recovered,new_recovered,active_cases,serious_critical,total_cases_per_million,deaths_per_million,total_tests,tests_per_million,population,continent,datetime
0,1,USA,"111,820,082",0,"1,219,487",0,"109,814,428",0,"786,167",940,"333,985","3,642","1,186,851,502","3,544,901","334,805,269",North America,2024-04-25
1,2,India,"45,035,393",0,"533,570",0,"32,016",0,0,379,"935,879,495","665,334",0,0,"1,406,631,776",Asia,2024-04-25
2,3,France,"40,138,560",0,"167,642",0,"39,970,918",0,0,"612,013","2,556","271,490,188","4,139,547",0,"65,584,518",Europe,2024-04-25
3,4,Germany,"38,828,995",0,"183,027",0,"38,240,600",0,"405,368","462,891","2,182","122,332,384","1,458,359",0,"83,883,596",Europe,2024-04-25
4,5,Brazil,"38,743,918",0,"711,380",0,"36,249,161",0,"1,783,377","179,908","3,303","63,776,166","296,146",0,"215,353,593",South America,2024-04-25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226,227,Tokelau,80,0,0,0,"58,055",0,80,0,0,0,0,0,"1,378",None,2024-04-25
227,228,Vatican City,29,0,0,0,29,0,0,"36,295",0,0,0,0,799,Europe,2024-04-25
228,229,Western Sahara,10,0,1,0,9,0,0,16,2,0,0,0,"626,161",Africa,2024-04-25
229,230,None,9,0,2,0,7,0,0,0,0,0,0,0,None,None,2024-04-25


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sb
import requests
import re
from bs4 import BeautifulSoup as bf
from datetime import datetime
url = 'https://www.worldometers.info/coronavirus/'
response = requests.get(url)

soup = bf(response.content, 'lxml')
"""with open('Coronavirus_24.html', mode = 'wb')as file:
    file.write(response.content)"""
bloc_countries = soup.find('table',class_="table table-bordered table-hover main_table_countries").contents[3]


#Id in tab 
id_case = bloc_countries.findAll('td', style="font-size:12px;color: grey;text-align:center;vertical-align:middle;")
id_list = [i.contents[0] for i in id_case]

#Country Cases
country_case = bloc_countries.findAll('td', style="font-weight: bold; font-size:15px; text-align:left;")
country_list = [i.contents[0].contents[0] for i in country_case]

#Total_Cases
total_cases_case = bloc_countries.findAll('td', style="font-weight: bold; text-align:right")
total_case_index = [i for i in range(len(total_cases_case)) if i%11 == 0]
list_searched = [total_cases_case[i] for i in total_case_index]
total_cases_list = [i.contents[0] for i in list_searched[:231]]


'''Fuction to convert the website into string 
and filter out contents since it's not possible effectively on some columns'''
#I'm obliged to add manually new cases in dataset , no way to extract directly content
def worldometers_to_string(bloc_countries):
    df_countries = []
    bloc_countries = str(bloc_countries)
    parts = re.findall(r'''(\n<td style="font-size:12px;color: grey;text-align:center;vertical-align:middle;">[\d]*</td>\n[\s\S]*?\n<!-- 1 Case/Death/Test every X -->\n)''',bloc_countries)
    print(len(parts))
    for i in range(len(parts)):
        #Id in tab
        Id_str = re.findall(r'''(grey;text-align:center;vertical-align:middle;">[\d,]+</td>)''',parts[i])
        Id_from_str = re.findall('[\d]+',Id_str[0])[0]
        #Id_from_str

        #Country in tab
        country_str = re.findall(r'''(<a class="mt_a" href="country/[a-zA-Z"/-]+>[a-zA-Z .]+</a>)''',parts[i])
        try:
            country_from_str = re.findall('[a-zA-Z .]+(?#</a>$)',country_str[0])[-2]
        except:
            country_from_str =None
        #country_from_str

        #Total_cases in tab
        total_cases_str = re.findall(r'''(<td style="font-weight: bold; text-align:right">[0-9,]+</td>)''',parts[i])
        try:
            total_cases_from_str = re.findall('[0-9,]+(?#</d>$)',total_cases_str[0])[0]
        except:
            total_cases_from_str = 0
        #total_cases_from_str
    
        #New_cases in tab
        new_cases_str = re.findall(r'''(text-align:right;background-color:#FFEEAA;">[0-9,+]+</td>)''',parts[i])
        try:
            new_cases_from_str = re.findall('[0-9,]+(?#</d>$)',new_cases_str[0])[0]
        except:
            new_cases_from_str = 0
        #new_cases_from_str

        #Total_Deaths in tab
        total_death_str = re.findall(r'''(<td style="font-weight: bold; text-align:right;">[0-9, ]+</td>)''',parts[i])
        try:
            total_death_from_str = re.findall('[0-9,]+(?#</d>$)',total_death_str[0])[0]
        except:
            total_death_from_str =0 
        #total_death_from_str

        #New_deaths in tab
        new_deaths_str = re.findall(r'''(background-color:red; color:white">[0-9,+]+</td>)''',parts[i])
        try:
            new_deaths_from_str = re.findall('[0-9,]+(?#</d>$)',new_deaths_str[0])[-1]
        except:
            new_deaths_from_str = 0
        #new_deaths_from_str

        #Total_Recovered in tab
        total_recovered_str = re.findall(r'''(<span style="color:grey; font-style: italic;">[0-9, ]+</span>)''',parts[i])
        #Affect the extraction of serious_critical cases
        if len(total_recovered_str) == 0:
            try:
                total_recovered_str = re.findall(r'''(<td style="font-weight: bold; text-align:right">[0-9, ]+</td>)''',parts[i])
                total_recovered_from_str = re.findall('[0-9,]+(?#</d>$)',total_recovered_str[1])[0]
            except:
                total_recovered_from_str=0
        else:
            total_recovered_from_str = re.findall('[0-9,]+(?#</d>$)',total_recovered_str[0])[0]
        #total_recovered_from_str

        #New_recovered in tab 
        new_recovered_str = re.findall(r'''(background-color:#c8e6c9; color:#000">[0-9,+]+</td>)''',parts[i])
        try:
            new_recovered_from_str = re.findall('[0-9,]+(?#</d>$)',new_recovered_str[0])[-1]
        except:
            new_recovered_from_str =0
        #new_recovered_from_str

        #Active_cases in tab 
        active_cases_str = re.findall(r'''(<td style="text-align:right;font-weight:bold;">[0-9,]+</td>)''',parts[i])
        try:
            active_cases_from_str = re.findall('[0-9,]+(?#</d>$)',active_cases_str[0])[-1]
        except:
            active_cases_from_str = 0
        #active_cases_from_str

        #Serious_critical, total_cases_per_million, deaths_per_million, tests_per_million in tab (based on this extraction method, depend of total recovered)
        total_recovered_str = re.findall(r'''(<span style="color:grey; font-style: italic;">[0-9, ]+</span>)''',parts[i])
        pattern_search = re.findall(r'''(<td style="font-weight: bold; text-align:right">[0-9,]+</td>)''',parts[i])
        serious_critical_str = re.findall(r'''(<td style="font-weight: bold; text-align:right">[0-9,]+</td>)''',parts[i])
        total_cases_per_million_str = pattern_search
        deaths_per_million_str = pattern_search
        total_tests_str = pattern_search
        tests_per_million_str = pattern_search
        if len(total_recovered_str) == 0:
            total_recovered_str = re.findall(r'''(<td style="font-weight: bold; text-align:right">[0-9, ]+</td>)''',parts[i])
            try:
                serious_critical_from_str = re.findall('[0-9,]+(?#</d>$)',serious_critical_str[2])[0]
            except:
                serious_critical_from_str = 0
            try:
                total_cases_per_million_from_str = re.findall('[0-9,]+(?#</d>$)',total_cases_per_million_str[3])[0]
            except:
                total_cases_per_million_from_str =0
            try:
                deaths_per_million_from_str = re.findall('[0-9,]+(?#</d>$)',total_cases_per_million_str[4])[0]
            except:
                deaths_per_million_from_str = 0
            try:
                total_tests_from_str = re.findall('[0-9,]+(?#</d>$)',total_cases_per_million_str[5])[0]
            except:
                total_tests_from_str = 0
            try:
                tests_per_million_from_str = re.findall('[0-9,]+(?#</d>$)',total_cases_per_million_str[6])[0]
            except:
                tests_per_million_from_str = 0
        else:
            try:
                serious_critical_from_str = re.findall('[0-9,]+(?#</d>$)',serious_critical_str[1])[0]
            except:
                serious_critical_from_str = 0
            try:
                total_cases_per_million_from_str = re.findall('[0-9,]+(?#</d>$)',total_cases_per_million_str[2])[0]
            except:
                total_cases_per_million_from_str =0
            try:
                deaths_per_million_from_str = re.findall('[0-9,]+(?#</d>$)',total_cases_per_million_str[3])[0]
            except:
                deaths_per_million_from_str = 0
            try:
                total_tests_from_str = re.findall('[0-9,]+(?#</d>$)',total_cases_per_million_str[4])[0]
            except:
                total_tests_from_str = 0
            try:
                tests_per_million_from_str = re.findall('[0-9,]+(?#</d>$)',total_cases_per_million_str[5])[0]
            except:
                tests_per_million_from_str = 0
    
            #Population in tab
        population_str = re.findall(r'''(text-align:right"><a href="/world-population/[a-zA-Z"/-]+>[0-9,]+</a>)''',parts[i])
        try :
            population_from_str = re.findall('[0-9,]+(?#</a>$)',population_str[0])[0]
        except:
            population_from_str = None
        #population_from_str

        #Continent in Worldometer tab sheet
        continent_str = re.findall(r'''(<td data-continent[a-zA-Z": =]+>[a-zA-Z. ]+</td>)''',parts[i])
        try:
            continent_from_str = re.findall('[a-zA-Z. ]+(?#</td>$)',continent_str[0])[-2]
        except:
            continent_from_str = None
        #continent_from_str
    

        #DataFrame building 
        df_countries.append({'Id':Id_from_str,
                         'country':country_from_str,
                         'total_cases':total_cases_from_str,
                         'new_cases':new_cases_from_str,
                         'total_death':total_death_from_str,
                         'new_deaths':new_deaths_from_str,
                         'total_recovered':total_recovered_from_str,
                         'new_recovered':new_recovered_from_str,
                         'active_cases':active_cases_from_str,
                         'serious_critical':serious_critical_from_str,
                         'total_cases_per_million':total_cases_per_million_from_str,
                         'deaths_per_million':deaths_per_million_from_str,
                         'total_tests':total_tests_from_str,
                         'tests_per_million':tests_per_million_from_str,
                         'population':population_from_str,
                         'continent':continent_from_str})

    df_coronavirus = pd.DataFrame(df_countries, columns = ['Id','country','total_cases','new_cases','total_death','new_deaths','total_recovered','new_recovered','active_cases','serious_critical','total_cases_per_million','deaths_per_million','total_tests','tests_per_million','population','continent'])
    return df_coronavirus

df = worldometers_to_string(bloc_countries)
tdy = datetime.today().strftime('%Y-%m-%d')
df['datetime'] = tdy

df.to_csv('Coronavirus_'+tdy+'.csv',index =False)